In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications import ResNet50, VGG16, VGG19, InceptionResNetV2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


In [10]:

img_height = 150
img_width = 150
batch_size = 32
data_directory = '/Users/baudi/AI/practicas/uvas/data/train_val/output/'
test_data_directory = '/Users/baudi/AI/practicas/uvas/data/test/output/'
val_split = 0.2
seed = 42
num_classes = 4
learning_rate = 0.00001


In [11]:

def adjustments(image):
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.9, 1.1)
    return image

def process_labels(images, labels):
    one_hot_labels = tf.one_hot(labels, depth=num_classes)
    return images, one_hot_labels

train_data_raw = image_dataset_from_directory(
    data_directory,
    validation_split=val_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_data_raw.class_names

train_data = train_data_raw.map(lambda x, y: (adjustments(x), y)).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)


val_data = image_dataset_from_directory(
    data_directory,
    validation_split=val_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)

test_data = image_dataset_from_directory(
    test_data_directory,
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)


Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 2000 files belonging to 4 classes.


In [ ]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_height, img_width, 3)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [ ]:
def build_model_ResNet50():
    base_model = ResNet50(
        include_top=False, 
        input_shape=(img_height, img_width, 3), 
        weights='imagenet'
    )

    # Obtener las últimas 10 capas
    x = base_model.layers[-10].output

    # Agregar las capas densas para la clasificación
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    # Crear el modelo final
    model = Model(inputs=base_model.input, outputs=output)

    # Congelar todas las capas de la red base excepto las últimas 10
    for layer in base_model.layers[:-10]:
        layer.trainable = False

    # Compilar el modelo
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    return model


In [ ]:
def build_model_VGG16():
    # Cargar la red pre-entrenada VGG16
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    
    # Congelar todas las capas de la red base excepto las últimas 10
    for layer in base_model.layers[:-10]:
        layer.trainable = False
        
    # Añadir capas adicionales para la clasificación
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    # Crear el modelo final
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compilar el modelo
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [5]:
def build_model_VGG19():
    # Cargar la red pre-entrenada VGG19
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    
    # Congelar todas las capas de la red base excepto las últimas 12
    for layer in base_model.layers[:-12]:
        layer.trainable = False
        
    # Añadir capas adicionales para la clasificación
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    # Crear el modelo final
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compilar el modelo
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [ ]:
def build_model_InceptionResNetV2():
    # Cargar la red pre-entrenada InceptionResNetV2
    base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

    # Congelar todas las capas de la red base excepto las últimas 12
    for layer in base_model.layers[:-12]:
        layer.trainable = False

    # Añadir capas adicionales para la clasificación
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    # Crear el modelo final
    model = Model(inputs=base_model.input, outputs=output)

    # Compilar el modelo
    model.compile(optimizer=SGD(learning_rate=learning_rate,momentum=0.9),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:

def cross_validate(model_builder, n_splits=2, epochs=20):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    X = np.concatenate([x for x, y in train_data] + [x for x, y in val_data])
    y = np.concatenate([y for x, y in train_data] + [y for x, y in val_data])

    accuracies = []
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y), start=1):
        model = model_builder()
        print(f"Comienza el entrenamiento del fold {fold}")
        history = model.fit(X[train_idx],
                  y[train_idx],
                  validation_data=(X[val_idx], y[val_idx]),
                  epochs=epochs)
        print(f"Termina el entrenamiento del fold {fold}")
        val_accuracy = history.history['val_accuracy']
        last_val_accuracy = val_accuracy[-1]
        print(f"Precisión de validación del fold {fold}: {last_val_accuracy}")

        accuracies.append(last_val_accuracy)

    return np.mean(accuracies)


In [ ]:

mean_accuracy = cross_validate(build_model_VGG19,5,20)
print(f'Mean accuracy: {mean_accuracy:.4f}')


In [8]:

def train_final_model(model_builder, epochs=20):
    X = np.concatenate([x for x, y in train_data] + [x for x, y in val_data])
    y = np.concatenate([y for x, y in train_data] + [y for x, y in val_data])

    final_model = model_builder()

    # Definir el callback de EarlyStopping
    early_stopping = EarlyStopping(monitor='loss', patience=3, min_delta=0.001)

    # Agregar el callback al método fit
    final_model.fit(X, y, epochs=epochs)

    return final_model

final_model = train_final_model(build_model_VGG19,30)


Epoch 1/30


2023-04-09 23:20:56.340155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 54s 213ms/step - loss: 1.0292 - accuracy: 0.6094
Epoch 2/30
250/250 [==============================] - 53s 211ms/step - loss: 0.2052 - accuracy: 0.9321
Epoch 3/30
250/250 [==============================] - 53s 213ms/step - loss: 0.1023 - accuracy: 0.9658
Epoch 4/30
250/250 [==============================] - 53s 213ms/step - loss: 0.0499 - accuracy: 0.9835
Epoch 5/30
250/250 [==============================] - 53s 213ms/step - loss: 0.0501 - accuracy: 0.9850
Epoch 6/30
250/250 [==============================] - 53s 211ms/step - loss: 0.0187 - accuracy: 0.9950
Epoch 7/30
250/250 [==============================] - 54s 215ms/step - loss: 0.0316 - accuracy: 0.9896
Epoch 8/30
250/250 [==============================] - 54s 217ms/step - loss: 0.0166 - accuracy: 0.9954
Epoch 9/30
250/250 [==============================] - 54s 217ms/step - loss: 0.0089 - accuracy: 0.9974
Epoch 10/30
250/250 [==============================] - 54s 215ms/step - loss: 0.0156

In [12]:
test_loss, test_accuracy = final_model.evaluate(test_data)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Y_pred = final_model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)

y_true = np.concatenate([y.numpy() for _, y in test_data.unbatch()])
y_true_labels = np.argmax(y_true.reshape(-1, len(class_names)), axis=1)

print('Classification Report:')
print(classification_report(y_true_labels, y_pred, target_names=class_names))

print('Confusion Matrix:')
print(confusion_matrix(y_true_labels, y_pred))


63/63 [==============================] - 7s 107ms/step - loss: 0.1396 - accuracy: 0.9770
Test Loss: 0.1396
Test Accuracy: 0.9770
63/63 [==============================] - 7s 106ms/step
Classification Report:
                                            precision    recall  f1-score   support

                         Grape___Black_rot       0.95      0.97      0.96       565
              Grape___Esca_(Black_Measles)       0.97      0.97      0.97       670
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)       1.00      0.99      1.00       547
                           Grape___healthy       1.00      1.00      1.00       218

                                  accuracy                           0.98      2000
                                 macro avg       0.98      0.98      0.98      2000
                              weighted avg       0.98      0.98      0.98      2000

Confusion Matrix:
[[546  18   1   0]
 [ 23 647   0   0]
 [  3   0 544   0]
 [  1   0   0 217]]
